In [16]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil



import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
#os.environ['TPU_LOAD_LIBRARY']='0'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
import metrics as metrics
from optimizers import *
import schedulers as schedulers

from scipy import stats

import enformer_vanilla as enformer


In [17]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-5')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy=\
        tf.data.experimental.AutoShardPolicy.OFF
    options.deterministic=False
    options.experimental_threading.max_intra_op_parallelism=1
    mixed_precision.set_global_policy('mixed_bfloat16')
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 1 # batch size 24, use LR ~ 2.5 e -04
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS
    

2022-12-24 21:29:11.108671: I tensorflow/core/common_runtime/eager/kernel_and_device.cc:94] Ignoring error status when releasing multi-device function handle UNIMPLEMENTED: Releasing a multi-device component handle on a remote device is not yet implemented.
2022-12-24 21:29:11.109463: I tensorflow/core/common_runtime/eager/kernel_and_device.cc:94] Ignoring error status when releasing multi-device function handle UNIMPLEMENTED: Releasing a multi-device component handle on a remote device is not yet implemented.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: node-5


INFO:tensorflow:Initializing the TPU system: node-5


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [18]:
def deserialize_val_TSS(serialized_example,input_length,max_shift, out_length,num_targets):
    """Deserialize bytes stored in TFRecordFile."""
    feature_map = {
        'sequence': tf.io.FixedLenFeature([], tf.string),
        'target': tf.io.FixedLenFeature([], tf.string),
        'tss_mask': tf.io.FixedLenFeature([], tf.string),
        'gene_name': tf.io.FixedLenFeature([], tf.string)
    }
    
    data = tf.io.parse_example(serialized_example, feature_map)

    shift = 5
    input_seq_length = input_length + max_shift
    interval_end = input_length + shift
    
    ### rev_comp
    #rev_comp = random.randrange(0,2)

    example = tf.io.parse_example(serialized_example, feature_map)
    sequence = tf.io.decode_raw(example['sequence'], tf.bool)
    sequence = tf.reshape(sequence, (input_length + max_shift, 4))
    sequence = tf.cast(sequence, tf.float32)
    sequence = tf.slice(sequence, [shift,0],[input_length,-1])
    
    target = tf.io.decode_raw(example['target'], tf.float16)
    target = tf.reshape(target,
                        (out_length, num_targets))
    
    tss_mask = tf.io.parse_tensor(data['tss_mask'],
                                  out_type=tf.int32)

    
    gene_name= tf.io.parse_tensor(example['gene_name'],out_type=tf.int32)
    gene_name = tf.tile(tf.expand_dims(gene_name,axis=0),[638])
    cell_types = tf.range(0,638)
    
    return {'sequence': tf.ensure_shape(sequence,
                                        [input_length,4]),
            'target': tf.ensure_shape(target,
                                      [896,num_targets]),
            'tss_mask': tf.ensure_shape(tss_mask,
                                        [896,1]),
            'gene_name': tf.ensure_shape(gene_name,
                                         [638,]),
            'cell_types': tf.ensure_shape(cell_types,
                                           [638,])}

In [19]:
pearsonsR = metrics.MetricDict({'PearsonR': metrics.PearsonR(reduce_axis=(0,1))})
R2 = metrics.MetricDict({'R2': metrics.R2(reduce_axis=(0,1))})


with strategy.scope():
    
    list_files_val = (tf.io.gfile.glob(os.path.join("gs://genformer_data/expanded_originals/196k/human/tfrecords_tss",
                                                "tssmask-valid*.tfr")))
    
    files = tf.data.Dataset.list_files(list_files_val)
    dataset_build = tf.data.TFRecordDataset(files,
                                      compression_type='ZLIB',
                                      num_parallel_reads=4)
    dataset_build = dataset_build.with_options(options)
    dataset_build = dataset_build.map(lambda record: deserialize_val_TSS(record,
                                                     196608,
                                                     10,
                                                     896,
                                                     5313),
                          deterministic=False,
                          num_parallel_calls=4)
    

    dataset_build=dataset_build.repeat(1).batch(1).prefetch(1)
    val_dist_build= strategy.experimental_distribute_dataset(dataset_build)
    val_dist_build_it = iter(val_dist_build)
    
    model = enformer.Enformer()
    
    def run_build_dist(iterator):
        @tf.function
        def run_build(inputs):
            sequence = inputs['sequence']
            model(sequence,is_training=False)
        strategy.run(run_build,
                     args=(next(iterator),))
    
    run_build_dist(val_dist_build_it)
    ### build model then load weights
    
    
    def run_val_dist(iterator):
        @tf.function
        def run_build(inputs):
            sequence = inputs['sequence']
            model(sequence,is_training=False)
        strategy.run(run_build,
                     args=(next(iterator),))
    
    checkpoint_options = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")
    checkpoint = tf.train.Checkpoint(module=model)#,options=options)
    tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
    latest = tf.train.latest_checkpoint("/home/jupyter/dev/BE_CD69_paper_2022/enformer_fine_tuning/checkpoint/sonnet_weights/")

    checkpoint.restore(latest,options=checkpoint_options).assert_consumed()
    

## next(val_dist_build_it)

In [21]:
with strategy.scope():
    overall_step = 0
    pearsons_list=[]
    r2_list=[]
    def dist_run_model(iterator):
        @tf.function
        def run_model(inputs):
            sequence = inputs['sequence']
            target = inputs['target'][:,:,4675:]
            
            tss_mask = inputs['tss_mask']
            gene_name = inputs['gene_name']
            
            cell_types = inputs['cell_types']
            return model(sequence,is_training=False)['human'][:,:,4675:],target,tss_mask,gene_name,cell_types
        
        dist_out,dist_target,tss_mask,gene_name,cell_types = strategy.run(run_model,
                                args=(next(iterator),))
        return dist_out,dist_target,tss_mask,gene_name,cell_types
    
    
    pred_gene_cents_all = []
    true_gene_cents_all = []
    gene_names_all = []
    cell_types_all = []
    
    for k,file in enumerate(list_files_val):
        if k > 0:
            continue

        dataset = tf.data.TFRecordDataset(file,
                                          compression_type='ZLIB',
                                          num_parallel_reads=4)
        dataset = dataset.with_options(options)
        dataset = dataset.map(lambda record: deserialize_val_TSS(record,
                                                         196608,
                                                         10,
                                                         896,
                                                         5313),
                              deterministic=False,
                              num_parallel_calls=4)

        dataset=dataset.repeat(1).batch(8).prefetch(1)
        val_dist= strategy.experimental_distribute_dataset(dataset)
        val_dist_it = iter(val_dist)

        for step in range(32):
            try:
                dist_out,dist_target,dist_tss_mask,dist_gene_name,dist_cell_types = dist_run_model(val_dist_it)
                
                for i in range(8):
                    dist_out_vals = dist_out.values[i]
                    dist_target_vals = dist_target.values[i]
                    dist_tss_mask_vals = dist_tss_mask.values[i]
                    dist_gene_name_vals = dist_gene_name.values[i]
                    dist_cell_types_vals = dist_cell_types.values[i]
                    
                    true_gene_cent = tf.reduce_sum((tf.cast(dist_target_vals,dtype=tf.float32) * tf.cast(dist_tss_mask_vals,dtype=tf.float32)),
                                                   axis=1)
                    pred_gene_cent = tf.reduce_sum(tf.cast(dist_out_vals,dtype=tf.float32) * tf.cast(dist_tss_mask_vals,dtype=tf.float32),
                                                   axis=1)
                    
                    true_gene_cents_all.append(true_gene_cent[0,:].numpy())
                    pred_gene_cents_all.append(pred_gene_cent[0,:].numpy())
                    gene_names_all.append(dist_gene_name_vals[0,:].numpy())
                    cell_types_all.append(dist_cell_types_vals[0,:].numpy())

                    overall_step += 1
                
            except StopIteration:
                print('reached end')
                break
        

In [26]:
import pandas as pd
from scipy.stats.stats import pearsonr, spearmanr
from scipy.stats import zscore
results_df = pd.DataFrame()
results_df['true'] = np.concatenate(true_gene_cents_all,axis=0)
results_df['pred'] = np.concatenate(pred_gene_cents_all,axis=0)
results_df['gene_encoding']  =np.concatenate(gene_names_all,axis=0)
results_df['cell_type_encoding'] = np.concatenate(cell_types_all,axis=0)

results_df=results_df.groupby(['gene_encoding', 'cell_type_encoding']).agg({'true': 'sum', 'pred': 'sum'})

results_df['true_zscore']=results_df.groupby(['cell_type_encoding']).true.transform(lambda x : zscore(x))
results_df['pred_zscore']=results_df.groupby(['cell_type_encoding']).pred.transform(lambda x : zscore(x))

true_zscore=results_df[['true_zscore']].to_numpy()[:,0]

pred_zscore=results_df[['pred_zscore']].to_numpy()[:,0]

cell_specific_corrs=results_df.groupby('cell_type_encoding')[['true_zscore','pred_zscore']].corr(method='pearson').unstack().iloc[:,1].tolist()

In [28]:
np.mean(cell_specific_corrs)

0.47793149458487727

In [30]:
results_df.groupby('cell_type_encoding')[['true_zscore','pred_zscore']].corr(method='pearson')

true_zscore  pred_zscore
cell_type_encoding                                      
0                  true_zscore     1.000000     0.136402
                   pred_zscore     0.136402     1.000000
1                  true_zscore     1.000000     0.076286
                   pred_zscore     0.076286     1.000000
2                  true_zscore     1.000000     0.149145
...                                     ...          ...
635                pred_zscore     0.320477     1.000000
636                true_zscore     1.000000     0.618767
                   pred_zscore     0.618767     1.000000
637                true_zscore     1.000000     0.659398
                   pred_zscore     0.659398     1.000000

[1276 rows x 2 columns]

In [7]:
import tensorflow as tf
def deserialize_val_TSS(serialized_example):
    """Deserialize bytes stored in TFRecordFile."""
    feature_map = {
        'sequence': tf.io.FixedLenFeature([], tf.string),
        'target': tf.io.FixedLenFeature([], tf.string)
    }
    
    example = tf.io.parse_example(serialized_example, feature_map)


    example = tf.io.parse_example(serialized_example, feature_map)
    sequence = tf.io.decode_raw(example['sequence'], tf.bool)
    sequence = tf.reshape(sequence, (393216 + 10, 4))
    sequence = tf.cast(sequence, tf.float32)
    sequence = tf.slice(sequence, [5,0],[393216,-1])
    
    target = tf.io.decode_raw(example['target'], tf.float16)
    target = tf.reshape(target,
                        (896, 5313))
    

    return {'sequence': tf.ensure_shape(sequence,
                                        [393216,4]),
            'target': tf.ensure_shape(target,
                                      [896,5313])}

dataset_build = tf.data.TFRecordDataset("gs://genformer_data/expanded_originals/393k/human/tfrecords/train-0-0.tfr",
                                  compression_type='ZLIB',
                                  num_parallel_reads=4)
#dataset_build = dataset_build.with_options(options)
dataset_build = dataset_build.map(lambda record: deserialize_val_TSS(record),
                      deterministic=False,
                      num_parallel_calls=4)


dataset_build=dataset_build.repeat(1).batch(1).prefetch(1)
#val_dist_build= strategy.experimental_distribute_dataset(dataset_build)
val_dist_build_it = iter(dataset_build)
out=next(val_dist_build_it)

In [11]:
out['sequence'][0,:25,:]

<tf.Tensor: shape=(25, 4), dtype=float32, numpy=
array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)>

In [6]:
import pandas as pd
import numpy as np

In [7]:
results_df = pd.read_csv('gene_level_predictions.tsv',sep='\t')

In [8]:
results_df=results_df.groupby(['gene_encoding', 'cell_type_encoding']).agg({'true': 'sum', 'pred': 'sum'})
results_df['true'] = np.log2(1.0+results_df['true'])
results_df['pred'] = np.log2(1.0+results_df['pred'])

try: 
    cell_specific_corrs=results_df.groupby('cell_type_encoding')[['true_zscore','pred_zscore']].corr(method='pearson').unstack().iloc[:,1].tolist()
except np.linalg.LinAlgError as err:
    cell_specific_corrs = [0.0] * len(np.unique(cell_types))

try: 
    gene_specific_corrs=results_df.groupby('gene_encoding')[['true_zscore','pred_zscore']].corr(method='pearson').unstack().iloc[:,1].tolist()
except np.linalg.LinAlgError as err:
    gene_specific_corrs = [0.0] * len(np.unique(gene_map))

corrs_overall = np.nanmean(cell_specific_corrs), \
                    np.nanmean(gene_specific_corrs)

KeyError: 'gene_encoding'

In [9]:
results_df

,true,pred,true_zscore,pred_zscore
0,8.473462,8.681625,0.853461,0.950524
1,8.804750,7.921510,1.303997,1.060832
2,8.827936,8.042851,1.261695,1.049663
3,7.849972,7.451152,1.178418,1.065244
4,7.515326,7.903860,0.888531,1.057324
...,...,...,...,...
1031641,6.978035,6.129662,1.834530,1.469072
1031642,7.423812,6.144497,1.659949,1.261396
1031643,6.003384,5.271635,1.465065,1.354486
1031644,8.865876,8.291864,1.403999,1.274530
